# Context-FID Score Presentation
## Necessary packages and functions call
# MamED expand：4 MamED
- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [11]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [14]:
iterations = 5
ori_data = np.load('../OUTPUT/energyED-new/samples/energy_norm_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/energyED-new/ddpm_fake_energyED-new.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [17]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.08128473036279533 

Iter 1:  context-fid = 0.06091078813671952 

Iter 2:  context-fid = 0.07328403445330293 

Iter 3:  context-fid = 0.0772785606351524 

Iter 4:  context-fid = 0.07789240275471335 

Final Score:  0.07413010326853671 ± 0.009830389909020664


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [18]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [20]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.7743710378108909 

Iter 1:  cross-correlation = 0.839591220204594 

Iter 2:  cross-correlation = 0.8638751562552264 

Iter 3:  cross-correlation = 0.7099737814721113 

Iter 4:  cross-correlation = 0.6703643368319112 

Final Score:  0.7716351065149467 ± 0.10237448749699332
